In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('Transforms').getOrCreate()

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 55714)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.12/socketserver.py", line 766, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 299, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 275, in accum_updates
    num_updates =

In [4]:
# collect_list

data = [('user1','book1'),
        ('user1','book2'),
        ('user2','book2'),
        ('user2','book4'),
        ('user3','book1')]

schema = 'user string, book string'

df_book = spark.createDataFrame(data,schema)

df_book.show()

df_book.groupBy('user').agg(collect_list('book')).show()

+-----+-----+
| user| book|
+-----+-----+
|user1|book1|
|user1|book2|
|user2|book2|
|user2|book4|
|user3|book1|
+-----+-----+

+-----+------------------+
| user|collect_list(book)|
+-----+------------------+
|user1|    [book1, book2]|
|user2|    [book2, book4]|
|user3|           [book1]|
+-----+------------------+



In [8]:
# PIVOT

df = spark.read.format('csv').option('inferSchema',True).option('header',True).load('BigMart_Sales.csv')

df.select(col('Item_Type'), col('Outlet_Size'), col('Item_MRP')).show()

df.groupBy('Item_Type').pivot('Outlet_Size').agg(avg('Item_MRP')).show()

+--------------------+-----------+--------+
|           Item_Type|Outlet_Size|Item_MRP|
+--------------------+-----------+--------+
|               Dairy|     Medium|249.8092|
|         Soft Drinks|     Medium| 48.2692|
|                Meat|     Medium| 141.618|
|Fruits and Vegeta...|       NULL| 182.095|
|           Household|       High| 53.8614|
|        Baking Goods|     Medium| 51.4008|
|         Snack Foods|       High| 57.6588|
|         Snack Foods|     Medium|107.7622|
|        Frozen Foods|       NULL| 96.9726|
|        Frozen Foods|       NULL|187.8214|
|Fruits and Vegeta...|     Medium| 45.5402|
|               Dairy|      Small|144.1102|
|Fruits and Vegeta...|     Medium|145.4786|
|         Snack Foods|      Small|119.6782|
|Fruits and Vegeta...|       High|196.4426|
|           Breakfast|      Small| 56.3614|
|  Health and Hygiene|     Medium|115.3492|
|           Breakfast|     Medium| 54.3614|
|         Hard Drinks|     Medium|113.2834|
|               Dairy|      Smal

In [13]:
# When-Otherwise

df = spark.read.format('csv').option('inferSchema',True).option('header',True).load('BigMart_Sales.csv')
df = df.withColumn('veg_flag',when(col('Item_Type')=='Meat','Non-Veg').otherwise('Veg'))

df.select(col('Item_Type'), col('veg_flag')).show()

df = df.withColumn('veg_exp_flag',when(((col('veg_flag')=='Veg') & (col('Item_MRP')<100)),'Veg_Inexpensive')\
                            .when((col('veg_flag')=='Veg') & (col('Item_MRP')>100),'Veg_Expensive')\
                            .otherwise('Non_Veg'))

df.select(col('Item_Type'), col('veg_flag'), col('veg_exp_flag')).show()

+--------------------+--------+
|           Item_Type|veg_flag|
+--------------------+--------+
|               Dairy|     Veg|
|         Soft Drinks|     Veg|
|                Meat| Non-Veg|
|Fruits and Vegeta...|     Veg|
|           Household|     Veg|
|        Baking Goods|     Veg|
|         Snack Foods|     Veg|
|         Snack Foods|     Veg|
|        Frozen Foods|     Veg|
|        Frozen Foods|     Veg|
|Fruits and Vegeta...|     Veg|
|               Dairy|     Veg|
|Fruits and Vegeta...|     Veg|
|         Snack Foods|     Veg|
|Fruits and Vegeta...|     Veg|
|           Breakfast|     Veg|
|  Health and Hygiene|     Veg|
|           Breakfast|     Veg|
|         Hard Drinks|     Veg|
|               Dairy|     Veg|
+--------------------+--------+
only showing top 20 rows
+--------------------+--------+---------------+
|           Item_Type|veg_flag|   veg_exp_flag|
+--------------------+--------+---------------+
|               Dairy|     Veg|  Veg_Expensive|
|         Soft 